In [1]:
%run ../../../ingestion/ingestion.ipynb
%run ../../../write/write.ipynb
%run ../../../alerts/webhook.ipynb

25/05/07 22:27:20 WARN Utils: Your hostname, lnb035652cps resolves to a loopback address: 127.0.1.1; using 192.168.1.172 instead (on interface wlp0s20f3)
25/05/07 22:27:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/07 22:27:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/07 22:27:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/05/07 22:27:21 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/05/07 22:27:22 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/05/07 22:27:22 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
# Cria uma instância da classe WebhookTeams utilizando o arquivo de configuração localizado em ../../config/teams.json
webhook = WebhookTeams("../../../config/teams.json")

In [ ]:
try:
    # Cria um leitor de Parquet
    reader = ParquetReader()
    
    # Executa a leitura do arquivo Parquet e retorna um DataFrame
    df = client_code(reader, "../../../layer/prata/saude/saude_leitos/*.parquet")
except:
    # Em caso de erro durante a leitura, envia uma notificação via Teams
    webhook.send_teams_notification("Erro de leitura!")



In [4]:
from pyspark.sql.functions import avg, round

# Calcula a média absoluta de leitos (leitos_total) para cada combinação de período e porte populacional.
# Esse cálculo não leva em conta a população total de cada grupo, apenas a média dos valores de leitos.
# A média é arredondada para duas casas decimais.
df_tratado = (
    df.groupBy("periodos", "porte_pop2020")
      .agg(round(avg("leitos_total"), 2).alias("media_leitos"))
      .orderBy("periodos", "media_leitos")
)


In [5]:
# Exibe as primeiras linhas do DataFrame tratado para visualização dos dados após as transformações
df_tratado.show()

+--------+--------------------+------------+
|periodos|       porte_pop2020|media_leitos|
+--------+--------------------+------------+
|    2010|           Até 5 mil|         0.9|
|    2010|Mais de 5 mil até...|        7.67|
|    2010|Mais de 10 mil at...|       27.48|
|    2010|Mais de 20 mil at...|       78.43|
|    2010|Mais de 50 mil at...|      167.98|
|    2010|Mais de 100 mil a...|      301.87|
|    2010|Mais de 200 mil a...|      558.56|
|    2010|Mais de 500 mil a...|      1678.0|
|    2010|    Mais de 1 milhão|    11426.67|
|    2011|           Até 5 mil|        0.93|
|    2011|Mais de 5 mil até...|        7.61|
|    2011|Mais de 10 mil at...|       27.51|
|    2011|Mais de 20 mil at...|       76.61|
|    2011|Mais de 50 mil at...|      167.82|
|    2011|Mais de 100 mil a...|       292.0|
|    2011|Mais de 200 mil a...|      559.16|
|    2011|Mais de 500 mil a...|     1666.33|
|    2011|    Mais de 1 milhão|    11369.67|
|    2012|           Até 5 mil|        0.93|
|    2012|

In [6]:
try:
    # Cria uma instância da classe responsável por exportar em formato Parquet
    exporter = ParquetExport()

    # Exporta o DataFrame para o caminho local especificado
    exporter.export(df, "../../../layer/ouro/saude/media_leitos_porte_populacional/", destination="local")
except:
    # Em caso de erro durante a exportação, envia uma notificação via Teams
    webhook.send_teams_notification("Erro de escrita!")